<center>
<h1>Assignment 1 : Reccommender Systems</h1>
<h3>Part 1 - Python Notebook </h3>
Duijn, Nicolette, s3026752<br>
Elangovan, Abhiram, s3442403 <br>
van Vliet, Nick, s3442403
</center>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold 
import math
from sklearn.metrics import mean_absolute_error as mae
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

#!pip install sweetviz
#import sweetviz as sv

In [ ]:
# Reading dataset
dataset = pd.read_table('ratings.dat', header = None, sep = '::', engine = 'python', usecols = [0,1,2], names = ('UserID','MovieID', 'Ratings'))

#eda = sv.analyze(dataset)
#eda.show_html()

In [ ]:
dataset.mean()

UserID     3024.512348
MovieID    1865.539898
Ratings       3.581564
dtype: float64

# Task 1.1 Naive Approaches

In [ ]:
# Make the cross validation

n_splits = 5
KF = KFold(n_splits=n_splits, random_state=123, shuffle=True)

# Create Lists 
GlobalAvergage_RMSE, UserAverage_RMSE, MovieAverage_RMSE, LinReg_RMSE, LinRegInter_RMSE = list(),list(),list(),list(),list()
GlobalAvergage_MAE, UserAverage_MAE, MovieAverage_MAE, LinReg_MAE, LinRegInter_MAE = list(),list(),list(),list(),list()

GlobalAvergage_RMSE_test, UserAverage_RMSE_test, MovieAverage_RMSE_test, LinReg_RMSE_test, LinRegInter_RMSE_test = list(),list(),list(),list(),list()
GlobalAvergage_MAE_test, UserAverage_MAE_test, MovieAverage_MAE_test, LinReg_MAE_test, LinRegInter_MAE_test = list(),list(),list(),list(),list()

In [ ]:
# Naive Approach - Global Average

start = time.time()

for train, test in KF.split(dataset):
    #Define X and y in train and test
    X_train, X_test = dataset.iloc[train, 0:2], dataset.iloc[test, 0:2]
    y_train, y_test = dataset.iloc[train, -1], dataset.iloc[test, -1]
    
    # Models
    GlobalAvergage = y_train.mean()

    # Predictions 
    pred_GlobalAvergage = [GlobalAvergage] * len(X_train) 
    pred_GlobalAvergage_test = [GlobalAvergage] * len(X_test)
   
    #RMSE's 
    RMSE_GlobalAvergage = math.sqrt(mean_squared_error(pred_GlobalAvergage,y_train))
    RMSE_GlobalAvergage_test = math.sqrt(mean_squared_error(pred_GlobalAvergage_test,y_test))

    GlobalAvergage_RMSE.append(RMSE_GlobalAvergage)
    GlobalAvergage_RMSE_test.append(RMSE_GlobalAvergage_test)

 
    # MAE's 
    MAE_GlobalAvergage = mae(pred_GlobalAvergage, y_train)
    MAE_GlobalAvergage_test = mae(pred_GlobalAvergage_test, y_test)

    
    GlobalAvergage_MAE.append(MAE_GlobalAvergage)
    GlobalAvergage_MAE_test.append(MAE_GlobalAvergage_test)
    
end = time.time()
tot_time = np.subtract(end, start)
print(f'Total runtime: {tot_time} s')    

Total runtime: 1.9274404048919678 s


In [ ]:
def User(Train, Test):
    predicted = list()
    user_dict = {}
    Global_mean = Train['Ratings'].mean()
    avg_user = Train.groupby("UserID").mean()['Ratings']
    keys = avg_user.index.values
    for i in keys:
        user_dict[i] = avg_user[i]
    
    for u, m, r in np.array(Test):
        if u in user_dict:
            predicted.append(user_dict[u])
        else:
            predicted.append(Global_mean)
    
    return(predicted)

def Movie(Train, Test):
    predicted = list()
    movie_dict = {}
    Global_mean = Train['Ratings'].mean()
    avg_movie = Train.groupby("MovieID").mean()['Ratings']
    keys = avg_movie.index.values
    for i in keys:
        movie_dict[i] = avg_movie[i]
    
    for u, m, r in np.array(Test):
        if m in movie_dict:
            predicted.append(movie_dict[m])
        else:
            predicted.append(Global_mean)
    return(predicted)

In [ ]:
# Naive Approach - User Average


start = time.time()

for train_indexes, test_indexes in KF.split(dataset):
    #Define train and test
    Train_set = dataset.iloc[train_indexes]
    Test_set = dataset.iloc[test_indexes]
    
    predicted_train = User(Train_set, Train_set)
    predicted_test = User(Train_set, Test_set)
    
    UserAverage_RMSE.append(np.sqrt(np.divide(np.sum(np.square(np.subtract(Train_set.iloc[:,2], predicted_train))), len(Train_set))))
    UserAverage_RMSE_test.append(np.sqrt(np.divide(np.sum(np.square(np.subtract(Test_set.iloc[:,2], predicted_test))), len(Test_set))))
    UserAverage_MAE.append(np.divide(np.sum(np.abs(np.subtract(Train_set.iloc[:,2], predicted_train))), len(Train_set)))
    UserAverage_MAE_test.append(np.divide(np.sum(np.abs(np.subtract(Test_set.iloc[:,2], predicted_test))), len(Test_set)))
end = time.time()
tot_time = np.subtract(end, start)
print(f'Total runtime: {tot_time} s')

Total runtime: 7.8140270709991455 s


In [ ]:
# Naive Approach - Movie Average


start = time.time()
for train_indexes, test_indexes in KF.split(dataset):
    #Define train and test
    Train_set = dataset.iloc[train_indexes]
    Test_set = dataset.iloc[test_indexes]
    
    predicted_train = Movie(Train_set, Train_set)
    predicted_test = Movie(Train_set, Test_set)
    
    MovieAverage_RMSE.append(np.sqrt(np.divide(np.sum(np.square(np.subtract(Train_set.iloc[:,2], predicted_train))), len(Train_set))))
    MovieAverage_RMSE_test.append(np.sqrt(np.divide(np.sum(np.square(np.subtract(Test_set.iloc[:,2], predicted_test))), len(Test_set))))
    MovieAverage_MAE.append(np.divide(np.sum(np.abs(np.subtract(Train_set.iloc[:,2], predicted_train))), len(Train_set)))
    MovieAverage_MAE_test.append(np.divide(np.sum(np.abs(np.subtract(Test_set.iloc[:,2], predicted_test))), len(Test_set)))

end = time.time()
tot_time = np.subtract(end, start)
print(f'Total runtime: {tot_time} s')

Total runtime: 7.889576435089111 s


In [ ]:
# Naive Approach - Linear Regression


start = time.time()


for train, test in KF.split(dataset):
    #Define X and y in train and test
    X_train, X_test = dataset.iloc[train, 0:2], dataset.iloc[test, 0:2]
    y_train, y_test = dataset.iloc[train, -1], dataset.iloc[test, -1]
    
    # Models
    UserAverage = dataset.iloc[train]
    MovieAverage = dataset.iloc[train]
    
    UserAverage_test = dataset.iloc[test]
    MovieAverage_test = dataset.iloc[test]
    
    # Create dataframe for Linear Regression
    UserAverage = UserAverage['Ratings'].groupby(UserAverage['UserID'], group_keys=False).transform('mean') 
    MovieAverage = MovieAverage['Ratings'].groupby(MovieAverage['MovieID'], group_keys=False).transform('mean')
    X_lr = pd.DataFrame(columns = ['UserAverage', 'MovieAverage'])
    X_lr['UserAverage'] = UserAverage
    X_lr['MovieAverage'] = MovieAverage       
    modelLinReg = linear_model.LinearRegression(fit_intercept=False).fit(X_lr,y_train)
    
    UserAverage_test = UserAverage_test['Ratings'].groupby(UserAverage_test['UserID'], group_keys=False).transform('mean') 
    MovieAverage_test = MovieAverage_test['Ratings'].groupby(MovieAverage_test['MovieID'], group_keys=False).transform('mean')
    X_lr_test = pd.DataFrame(columns = ['UserAverage', 'MovieAverage'])
    X_lr_test['UserAverage'] = UserAverage_test
    X_lr_test['MovieAverage'] = MovieAverage_test           
    
    # Predictions 
    pred_LinReg = modelLinReg.predict(X_lr)
    pred_LinReg_test = modelLinReg.predict(X_lr_test)
    
    #RMSE's 
    RMSE_LinReg = math.sqrt(mean_squared_error(pred_LinReg,y_train))
    RMSE_LinReg_test = math.sqrt(mean_squared_error(pred_LinReg_test,y_test))
    
    LinReg_RMSE.append(RMSE_LinReg)
    LinReg_RMSE_test.append(RMSE_LinReg_test)
    
    # MAE's 
    MAE_LinReg = mae(pred_LinReg, np.asarray(y_train))
    MAE_LinReg_test = mae(pred_LinReg_test, y_test)

    LinReg_MAE.append(MAE_LinReg)
    LinReg_MAE_test.append(MAE_LinReg_test)
    

end = time.time()
tot_time = np.subtract(end, start)
print(f'Total runtime: {tot_time} s')

Total runtime: 2.344867706298828 s


In [ ]:
# Naive Approach - Linear Regression with Intercept


start = time.time()


for train, test in KF.split(dataset):
    #Define X and y in train and test
    X_train, X_test = dataset.iloc[train, 0:2], dataset.iloc[test, 0:2]
    y_train, y_test = dataset.iloc[train, -1], dataset.iloc[test, -1]
    
    # Models
    UserAverage = dataset.iloc[train]
    MovieAverage = dataset.iloc[train]
    
    UserAverage_test = dataset.iloc[test]
    MovieAverage_test = dataset.iloc[test]
    
    # Create dataframe for Linear Regression
    UserAverage = UserAverage['Ratings'].groupby(UserAverage['UserID'], group_keys=False).transform('mean') 
    MovieAverage = MovieAverage['Ratings'].groupby(MovieAverage['MovieID'], group_keys=False).transform('mean')
    X_lr_inter = pd.DataFrame(columns = ['UserAverage', 'MovieAverage'])
    X_lr_inter['UserAverage'] = UserAverage
    X_lr_inter['MovieAverage'] = MovieAverage       
    modelLinRegInter = linear_model.LinearRegression(fit_intercept=True).fit(X_lr_inter,y_train)
    
    UserAverage_test = UserAverage_test['Ratings'].groupby(UserAverage_test['UserID'], group_keys=False).transform('mean') 
    MovieAverage_test = MovieAverage_test['Ratings'].groupby(MovieAverage_test['MovieID'], group_keys=False).transform('mean')
    X_lr_test_inter = pd.DataFrame(columns = ['UserAverage', 'MovieAverage'])
    X_lr_test_inter['UserAverage'] = UserAverage_test
    X_lr_test_inter['MovieAverage'] = MovieAverage_test       
    
    # Predictions 
    pred_LinRegInter = modelLinRegInter.predict(X_lr_inter)
    pred_LinRegInter_test = modelLinRegInter.predict(X_lr_test_inter)
    
    #RMSE's 
    RMSE_LinRegInter = math.sqrt(mean_squared_error(pred_LinRegInter,y_train))
    RMSE_LinRegInter_test = math.sqrt(mean_squared_error(pred_LinRegInter_test,y_test))
    
    LinRegInter_RMSE.append(RMSE_LinRegInter)
    LinRegInter_RMSE_test.append(RMSE_LinRegInter_test)
    
    # MAE's 
    MAE_LinRegInter = mae(pred_LinRegInter, y_train)
    MAE_LinRegInter_test = mae(pred_LinRegInter_test, y_test)

    LinRegInter_MAE.append(MAE_LinRegInter)
    LinRegInter_MAE_test.append(MAE_LinRegInter_test)
        

end = time.time()
tot_time = np.subtract(end, start)
print(f'Total runtime: {tot_time} s')

Total runtime: 2.323260545730591 s


In [ ]:
Average_RMSE_GlobalAvergage = np.mean(GlobalAvergage_RMSE)
Average_MAE_GlobalAvergage =  np.mean(GlobalAvergage_MAE)

Average_RMSE_UserAverage = np.mean(UserAverage_RMSE)
Average_MAE_UserAverage =  np.mean(UserAverage_MAE)

Average_RMSE_MovieAverage = np.mean(MovieAverage_RMSE)
Average_MAE_MovieAverage = np.mean(MovieAverage_MAE)

Average_RMSE_LinReg = np.mean(LinReg_RMSE)
Average_MAE_LinReg = np.mean(LinReg_MAE)

Average_RMSE_LinRegInter = np.mean(LinRegInter_RMSE)
Average_MAE_LinRegInter = np.mean(LinRegInter_MAE)

In [ ]:
print(f'Global Average on the Train set the mean RMSE                   = {Average_RMSE_GlobalAvergage}; the mean MAE = {Average_MAE_GlobalAvergage}')
print(f'User Average on the Train set the mean RMSE                     = {Average_RMSE_UserAverage}; the mean MAE = {Average_MAE_UserAverage}')
print(f'Movie Average on the Train set the mean RMSE                    = {Average_RMSE_MovieAverage}; the mean MAE = {Average_MAE_MovieAverage}')
print(f'Linear Regression on the Train set the mean RMSE                = {Average_RMSE_LinReg}; the mean MAE = {Average_MAE_LinReg}')
print(f'Linear Regression with intercept on the Train set the mean RMSE = {Average_RMSE_LinRegInter}; the mean MAE = {Average_MAE_LinRegInter}')

Global Average on the Train set the mean RMSE                   = 1.1171012202724344; the mean MAE = 0.9338608368374448
User Average on the Train set the mean RMSE                     = 1.0276690855268948; the mean MAE = 0.8227249188594131
Movie Average on the Train set the mean RMSE                    = 0.9742247376329038; the mean MAE = 0.7783327744410681
Linear Regression on the Train set the mean RMSE                = 0.9465615551888543; the mean MAE = 0.7585533811464578
Linear Regression with intercept on the Train set the mean RMSE = 0.9146271443996492; the mean MAE = 0.7250569615019544


In [ ]:
Average_RMSE_GlobalAvergage_test = np.mean(GlobalAvergage_RMSE_test)
Average_MAE_GlobalAvergage_test =  np.mean(GlobalAvergage_MAE_test)

Average_RMSE_UserAverage_test = np.mean(UserAverage_RMSE_test)
Average_MAE_UserAverage_test =  np.mean(UserAverage_MAE_test)

Average_RMSE_MovieAverage_test = np.mean(MovieAverage_RMSE_test)
Average_MAE_MovieAverage_test = np.mean(MovieAverage_MAE_test)

Average_RMSE_LinReg_test = np.mean(LinReg_RMSE_test)
Average_MAE_LinReg_test = np.mean(LinReg_MAE_test)

Average_RMSE_LinRegInter_test = np.mean(LinRegInter_RMSE_test)
Average_MAE_LinRegInter_test = np.mean(LinRegInter_MAE_test)

In [ ]:
print(f'Global Average on the Test set the mean RMSE                   = {Average_RMSE_GlobalAvergage_test}; the mean MAE = {Average_MAE_GlobalAvergage_test}')
print(f'User Average on the Test set the mean RMSE                     = {Average_RMSE_UserAverage_test}; the mean MAE = {Average_MAE_UserAverage_test}')
print(f'Movie Average on the Test set the mean RMSE                    = {Average_RMSE_MovieAverage_test}; the mean MAE = {Average_MAE_MovieAverage_test}')
print(f'Linear Regression on the Test set the mean RMSE                = {Average_RMSE_LinReg_test}; the mean MAE = {Average_MAE_LinReg_test}')
print(f'Linear Regression with intercept on the Test set the mean RMSE = {Average_RMSE_LinRegInter_test}; the mean MAE = {Average_MAE_LinRegInter_test}')

Global Average on the Test set the mean RMSE                   = 1.117100689212873; the mean MAE = 0.9338609691154162
User Average on the Test set the mean RMSE                     = 1.03551276240495; the mean MAE = 0.8290272360320732
Movie Average on the Test set the mean RMSE                    = 0.9794113512327757; the mean MAE = 0.7822917257566733
Linear Regression on the Test set the mean RMSE                = 0.934633421514661; the mean MAE = 0.7487582102802492
Linear Regression with intercept on the Test set the mean RMSE = 0.9005467579253942; the mean MAE = 0.7130920759240753


# Task 1.2 UV Matrix Decomposition


In [ ]:
dataset_task2 = dataset.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
)

In [ ]:

start = time.time()

# Initialize parameters
fig, (ax1, ax2) = plt.subplots(2)
n_iter = 0
rme_list = list()
MAE_list = list()
n_splits = 5
Threshold = 0.0001
np.random.seed(123)
rand_seeds = np.random.randint(10, size = 5)

# Cross validation 
KF = KFold(n_splits=5, random_state=123, shuffle=True)

# Kfold loop (5 datasets)
for Train, Test in KF.split(dataset):
    dataset_2_Train, dataset_2_Test = dataset.loc[Train], dataset.loc[Test]

    # Pivot Dataset and scale the dataset
    dataset_split = dataset_2_Train.pivot(
    index='UserID',
    columns='MovieID',
    values='Ratings'
    )
    scaler = StandardScaler(with_std=False)
    dataset_split = scaler.fit_transform(dataset_split)
    dataset_split = pd.DataFrame(dataset_split)

    #Set random seeds
    np.random.seed(rand_seeds[n_iter])
    #Create Init matrices
    M = dataset_split.to_numpy()
    d = 2
    n = dataset_split.shape[0]
    m = dataset_split.shape[1]
    a = dataset_split.stack().mean()
    U = np.empty([n,d])
    V = np.empty([d,m])
    V = np.random.randn(d, m) # random numbers to increase chance of reachine global minimum
    U = np.random.randn(n, d)
    uv = np.matmul(U,V)

    # Create Init KFold dependant parameters
    halt = True
    n_iter += 1
    n_loop = 1
    l_RMSE_loop = list()
    l_MAE_loop = list()

    # Calculate initial RMSE
    dif_squared_0 = np.nan_to_num(np.square(np.subtract(uv, M)))
    dif_squared_total_sum = np.sum(dif_squared_0)
    N_non_0 = np.count_nonzero(dif_squared_0)
    RME = np.divide(dif_squared_total_sum, N_non_0)
    RMSE = np.sqrt(RME)
    l_RMSE_loop.append(RMSE)

    # Calculate initial MAE
    dif_abs = np.nan_to_num(np.abs(np.subtract(uv, M)))
    dif_abs_sum = np.sum(dif_abs)
    N_non_0_abs = np.count_nonzero(dif_abs)
    MAE = np.divide(dif_abs_sum, N_non_0_abs)
    l_MAE_loop.append(MAE)

    # Iterate as long as change of RMSE is bigger than threshold
    while halt:
        RME_old = RMSE
        n_loop+=1

        #Update U matrix
        for r in range(U.shape[0]):
            numerator = 0
            denominator = 0

            for s in range(d):
                U_rk = U[r,:]
                U_rk = np.delete(U_rk, s, 0)
                V_kj = np.delete(V, s, 0)
                V_sj = V[s,:]
                P = np.matmul(U_rk, V_kj)
                m_rj = M[r,:]
                numerator = np.multiply(V_sj,(np.subtract(m_rj, P)))
                numerator = np.nansum(np.multiply(numerator, (np.isfinite(m_rj))))
                denominator = np.square(V_sj)
                denominator = np.nansum(np.multiply(denominator, (np.isfinite(m_rj))))
                new_var = np.divide(numerator, denominator)
                U[r,s] = new_var

        #update V matrix
        for r in range(d):
            for s in range(V.shape[1]):
                V_ks = V[:,s]
                V_ks = np.delete(V_ks, r, 0)
                U_ik = np.delete(U, r, 1)
                U_ir = U[:,r]
                P = np.matmul(U_ik, V_ks)
                m_is = M[:,s]
                numerator = np.multiply(U_ir, (np.subtract(m_is, P)))
                numerator = np.nansum(np.multiply(numerator, (np.isfinite(m_is))))
                denominator = np.square(U_ir)
                denominator = np.nansum(np.multiply(denominator, (np.isfinite(m_is))))
                newvar = np.divide(numerator, denominator)
                V[r,s] = newvar

        # Matrix multiplication and comparison to original matrix M + calc MSE
        uv = np.matmul(U,V)

        # Scale back to original matrix, check if values are 1 < value < 5
        # Set values outside range to 1 or 5 and scale back
        uv = scaler.inverse_transform(uv)
        uv[uv > 5] = 5
        uv[uv < 1 ] = 1
        uv = scaler.transform(uv)
        
        # Calculate RMSE and append to list
        dif_squared_0 = np.nan_to_num(np.square(np.subtract(uv, M)))
        dif_squared_total_sum = np.sum(dif_squared_0)
        N_non_0 = np.count_nonzero(dif_squared_0)
        RME = np.divide(dif_squared_total_sum, N_non_0)
        RMSE = np.sqrt(RME)
        halt = np.abs(RMSE - RME_old) > Threshold
        l_RMSE_loop.append(RMSE)

        # Calculate MAE and append to list
        dif_abs = np.nan_to_num(np.abs(np.subtract(uv, M)))
        dif_abs_sum = np.sum(dif_abs)
        N_non_0_abs = np.count_nonzero(dif_abs)
        MAE = np.divide(dif_abs_sum, N_non_0_abs)
        l_MAE_loop.append(MAE)

    # Print results of each kfold iteration
    print(f'fold {n_iter}; RMSE = {RMSE}')
    print(f'fold {n_iter}; MAE = {MAE}')
    rme_list.append(RMSE)
    MAE_list.append(MAE)
    uv_final = scaler.inverse_transform(uv)
    ax1.plot(range(n_loop), l_RMSE_loop, label = f'Iteration Kfold: {n_iter}')
    ax2.plot(range(n_loop), l_MAE_loop, label = f'Iteration Kfold: {n_iter}')

# Summarize final stats and create plots
rme_list_mean = np.mean(rme_list)
MAE_list_mean = np.mean(MAE_list)
print(f'Average RMSE over {n_iter} folds = {rme_list_mean}')
print(f'Average MAE over {n_iter} folds = {MAE_list_mean}')
plt.suptitle("UV Matrix decomposition")
plt.xlabel("Number of loops")
ax1.set_ylabel("RMSE")
ax1.legend()
ax2.set_ylabel("MAE")
ax2.legend()
plt.savefig("Figure.png")
plt.show()
ende = time.time()
print('Total runtime = ', (ende - start))

# Task 1.3 - Matrix Factorization

In [ ]:



def MatrixFactorization(data, num_factors, num_iter, regularization, learn_rate, num_folds):
    start = time.time()
    
    
    # Cross validation
    
    # percentage train data (5 fold = 80% train data)
    num_folds = num_folds
    
    # Create RMSE for train and test for every fold
    RMSE_fold_train = [0] * num_folds
    RMSE_fold_test = [0] * num_folds
    MAE_fold_train = [0] * num_folds
    MAE_fold_test = [0] * num_folds


    sequence_of_data = [x % num_folds for x in range(len(data))]
    np.random.shuffle(sequence_of_data)
    data_matrix = data.pivot(index='UserID',    columns='MovieID',    values='Ratings').fillna(0)
    data_matrix = np.asarray(data_matrix)

    for fold in range(num_folds):
        
        train_index = np.array([x != fold for x in sequence_of_data])
        test_index = np.array([x == fold for x in sequence_of_data])
        train = data[train_index]
        test = data[test_index]
                
        # make an array of the data
        #train_array = np.array(train)
        #test_array = np.array(test)
        train_matrix = train.pivot(index='UserID',    columns='MovieID',    values='Ratings').fillna(0)
        test_matrix = test.pivot(index='UserID',    columns='MovieID',    values='Ratings').fillna(0)
        train = np.asarray(train)
        test = np.asarray(test)
        train_matrix = np.asarray(train_matrix)
        test_matrix = np.asarray(test_matrix)
        
        # Specifiy the I and J for the matrices 
        I = data_matrix.shape[0]
        J = data_matrix.shape[1]
        K = num_factors
        
        # Initialize random weights
        U = np.random.rand(I, K)
        M = np.random.rand(K, J)
                
        # Create empty list for train and test
        RMSE_list_train = []
        RMSE_list_test = []
        MAE_list_train = []
        MAE_list_test = []
        
        for iter in range(num_iter):
            e_ij2 = 0
            e_ij2_test = 0
            abs_e_ij2 = 0
            abs_e_ij2_test = 0
            # if the value in the matrix is higher than 0, for every i and j
            for i in range(len(train_matrix)):
                for j in range(train_matrix.shape[1]):
                    if train_matrix[i][j]>0:
                      
                      # calculate the prediction with the intiliazed weights
                        pred = np.dot(U[i,:], M[:,j])

                        # Keep ratings between 1 and 5
                        if pred < 1:
                            pred = 1
                        elif pred > 5:
                            pred = 5

                        # Calculate the error 
                        e_ij = train_matrix[i][j] - pred
                        e_ij2 += (e_ij)**2
                        abs_e_ij2 += np.abs(e_ij)


                        #Update the weights
                        for k in range(num_factors):
                            grad_eij_u = -2 * e_ij * M[k][j]
                            grad_eij_m = -2 * e_ij * U[i][k]

                            U[i][k] = U[i][k] + learn_rate * ( - grad_eij_u - regularization * U[i][k] )
                            M[k][j] = M[k][j] + learn_rate * ( - grad_eij_m - regularization * M[k][j] )


            # N: the number of known values in the Matrix
            N = np.count_nonzero(train_matrix)
            RMSE_iter_train = np.sqrt(e_ij2 / N)
            print('Train RMSE:', {RMSE_iter_train})
            MAE_iter_train = np.sqrt(abs_e_ij2/N)
            print('Train MAE:', {MAE_iter_train})

            RMSE_list_train.append(RMSE_iter_train)
            MAE_list_train.append(MAE_iter_train)

            

            
            # if the value in the matrix is higher than 0, for every i and j Test
            for i in range(len(test_matrix)):
                for j in range(test_matrix.shape[1]):
                    if test_matrix[i][j]>0:
                        # calculate the prediction with the weights Test
                        pred = np.dot(U[i,:], M[:,j])

                        # Keep ratings between 1 and 5
                        if pred < 1:
                            pred = 1
                        elif pred > 5:
                            pred = 5

                        # Calculate the error Test
                        e_ij_test = test_matrix[i][j] - pred
                        e_ij2_test += (e_ij_test)**2
                        abs_e_ij2_test += np.abs(e_ij_test)
                            
            # N: the number of known values in the Matrix
            N_test = np.count_nonzero(test_matrix)
            RMSE_iter_test = np.sqrt(e_ij2_test / N_test)
            MAE_iter_test = np.sqrt(abs_e_ij2_test/N_test)
            print('Test RMSE:', {RMSE_iter_test})
            print('Test MAE:', {MAE_iter_test})

            RMSE_list_test.append(RMSE_iter_test)
            MAE_list_test.append(MAE_iter_test)
            
            # Only continue if the new RMSE is better than the previous RMSE
            if len(RMSE_list_train) >= 2 or len(RMSE_list_train) >= 2:
                if RMSE_list_train[-2] <= RMSE_list_train[-1]:
                    break
                elif RMSE_list_test[-2] <= RMSE_list_test[-1]:
                    break
        
            
        RMSE_fold_train[fold] = RMSE_list_train[-1]
        RMSE_fold_test[fold] = RMSE_list_test[-1]
        MAE_fold_train[fold] = MAE_list_train[-1]
        MAE_fold_test[fold] = MAE_list_test[-1]


        print('List RMSE train fold:', RMSE_fold_train)
        print('List RMSE test fold:',RMSE_fold_test)
        print('List MAE train fold:',MAE_fold_train)
        print('List MAE test fold:',MAE_fold_test)
        
    print('List RMSE train average over folds:', np.mean(RMSE_fold_train))
    print('List RMSE test average over folds:',np.mean(RMSE_fold_test))
    print('List MAE train average over folds:',np.mean(MAE_fold_train))
    print('List MAE test average over folds:',np.mean(MAE_fold_test))
    
    end = time.time()
    print('Total runtime = ', (end - start))
    return U, M

In [ ]:
nU, nM = MatrixFactorization(dataset, num_factors=10, num_iter=75, regularization=0.05, learn_rate=0.005, num_folds = 5)
nU, nM

Train RMSE: {0.9894672620439152}
Train MAE: {0.8875949645370108}
Test RMSE: {1.3391849143728503}
Test MAE: {1.0417030307954747}
Train RMSE: {0.9244729836517905}
Train MAE: {0.855091983099799}
Test RMSE: {1.3303845029477117}
Test MAE: {1.0407627503904322}
Train RMSE: {0.9141270079044284}
Train MAE: {0.8505508703227754}
Test RMSE: {1.3282400537386756}
Test MAE: {1.0396830372341999}
Train RMSE: {0.908572019210516}
Train MAE: {0.8479926656595218}
Test RMSE: {1.3278869718319641}
Test MAE: {1.0390611909739205}
Train RMSE: {0.9018264834853097}
Train MAE: {0.8448006588361568}
Test RMSE: {1.3286035397335192}
Test MAE: {1.0389785328197554}
List RMSE train fold: [0.9018264834853097, 0, 0, 0, 0]
List RMSE test fold: [1.3286035397335192, 0, 0, 0, 0]
List MAE train fold: [0.8448006588361568, 0, 0, 0, 0]
List MAE test fold: [1.0389785328197554, 0, 0, 0, 0]
Train RMSE: {0.9892615116796855}
Train MAE: {0.8877320617054213}
Test RMSE: {1.2871083593645565}
Test MAE: {1.0192746762842309}
Train RMSE: {0.923

(array([[0.27622676, 0.39552885, 0.99448094, ..., 0.91788872, 0.296852  ,
         0.36362695],
        [0.37518397, 0.48302718, 0.33916604, ..., 0.67601689, 0.84799373,
         0.88441672],
        [0.55351533, 0.81033752, 0.59466846, ..., 0.69230784, 0.29221977,
         0.57197859],
        ...,
        [0.17021586, 0.60183176, 0.66820459, ..., 0.94837288, 0.41419559,
         0.25188205],
        [0.83085445, 0.27953943, 0.44943221, ..., 0.55042074, 0.42048368,
         0.6309695 ],
        [0.6922269 , 0.18485851, 0.72635442, ..., 0.42562766, 0.85070567,
         0.21859602]]),
 array([[0.72615115, 0.64560252, 0.52830012, ..., 0.33906815, 0.20029138,
         0.0031049 ],
        [0.84463282, 0.51316129, 0.66858667, ..., 0.01243517, 0.72859083,
         0.78086475],
        [0.72370951, 0.46565335, 0.55312215, ..., 0.61168912, 0.68084409,
         0.05093988],
        ...,
        [0.7203731 , 0.59437689, 0.62502664, ..., 0.35935173, 0.92575971,
         0.4660372 ],
        [0.7

In [ ]:
nR = np.dot(nU,nM)
nR

array([[4.24539181, 3.39089687, 2.94274632, ..., 2.00717544, 4.25218181,
        2.41992202],
       [3.68452099, 2.96518659, 2.60612279, ..., 1.76420998, 3.15970766,
        2.44786765],
       [4.36718762, 3.49463951, 3.16066478, ..., 2.21020208, 3.76204481,
        2.8342062 ],
       ...,
       [4.0043966 , 3.01270271, 2.78670111, ..., 1.96401138, 3.72611663,
        2.76496634],
       [4.00490564, 3.26168165, 2.77181764, ..., 2.09660685, 3.27266224,
        2.31745337],
       [4.04174353, 3.03395036, 2.65875133, ..., 1.95709866, 3.7587383 ,
        2.26177498]])

In [ ]:
nM

array([[0.72615115, 0.64560252, 0.52830012, ..., 0.33906815, 0.20029138,
        0.0031049 ],
       [0.84463282, 0.51316129, 0.66858667, ..., 0.01243517, 0.72859083,
        0.78086475],
       [0.72370951, 0.46565335, 0.55312215, ..., 0.61168912, 0.68084409,
        0.05093988],
       ...,
       [0.7203731 , 0.59437689, 0.62502664, ..., 0.35935173, 0.92575971,
        0.4660372 ],
       [0.73502847, 0.24269081, 0.25560701, ..., 0.04902011, 0.63265336,
        0.72960115],
       [0.36248298, 0.79633972, 0.65408375, ..., 0.69378117, 0.20577234,
        0.09635716]])

In [ ]:
pd.DataFrame(nU).to_csv('Usermatrix.csv')  
pd.DataFrame(nM).to_csv('Moviematrix.csv')  
#pd.DataFrame(nR).to_csv('total_matrix.csv')  